## Limpieza 5: SkLearn

### A lo largo de este ejercicio de pair programming vamos a intentar eliminar los valores nulos de nuestras columnas. En la lección hemos aprendido varios métodos de skelarn intentemos aplicarlos todos. Manos a la obra!

In [172]:
import pandas as pd
import sidetable as stb
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

In [173]:
df = pd.read_csv("datos/Limpieza-3.csv", index_col = 0)
df.head()

,year,type,country,activity,age,species,month,fatal,sex
0,2018,Boating,usa,Paddling,57.0,White Shark,Jun,N,F
1,2018,Unprovoked,brazil,Swimming,18.0,Tiger Shark,Jun,Y,M
2,2018,Unprovoked,usa,Walking,15.0,Bull Shark,May,N,M
3,2018,Provoked,australia,Feeding sharks,32.0,Grey Shark,May,N,M
4,2018,Invalid,england,Fishing,21.0,Unspecified,May,N,M


In [208]:
#generamos dos copias del df:
df2 = df.copy()
df3 = df.copy()

Exploramos los casos nulos de cada columna del DataFrame:

In [175]:
df.stb.missing()

,missing,total,percent
month,181,1672,10.825359
age,158,1672,9.449761
species,126,1672,7.535885
fatal,99,1672,5.921053
activity,31,1672,1.854067
sex,14,1672,0.837321
country,10,1672,0.598086
year,0,1672,0.000000
type,0,1672,0.000000


In [176]:
#vamos a explorar los casos nulos de cada columna:
for col in df:
    if df[col].isnull().sum() != 0:
        print(f'La columna {col} tiene {df[col].isnull().sum()} casos nulos.')
    else:
        print(f'La columna {col} no tiene casos nulos.')

La columna year no tiene casos nulos.
La columna type no tiene casos nulos.
La columna country tiene 10 casos nulos.
La columna activity tiene 31 casos nulos.
La columna age tiene 158 casos nulos.
La columna species tiene 126 casos nulos.
La columna month tiene 181 casos nulos.
La columna fatal tiene 99 casos nulos.
La columna sex tiene 14 casos nulos.


### 1. Es el momento de eliminar los nulos:
- Reemplazad los valores nulos de la columna age por la media de la edad usando el método SimpleImputer.

In [177]:
for col in df.select_dtypes(include=np.number):
#vamos a comprobar cuántos nulos tiene la columna "sex"
    print(f'La columna {col} tiene {df[col].isnull().sum()} casos nulos.')
    #la distribución de hombres y mujeres:
    #y calcularemos la media:
    print(f'La media de la columna {col} es {round(df[col].mean())}.')
    print('-----------------------------------')

La columna year tiene 0 casos nulos.
La media de la columna year es 1991.
-----------------------------------
La columna age tiene 158 casos nulos.
La media de la columna age es 29.
-----------------------------------


In [178]:
def simple_imputer(df, col, estadis):
    imputer = SimpleImputer(strategy = estadis, missing_values = np.nan, copy = False)
    imputer = imputer.fit(df[[col]])
    df[col] = imputer.transform(df[[col]])
    return df

In [179]:
df = simple_imputer(df, "age", "mean")

In [180]:
#ahora, cuántos nulos quedan después de aplicar la función:
for col in df.select_dtypes(include=np.number):
    if df.select_dtypes(include=np.number)[col].isnull().sum() == 0:
        print(f'En la columna {col} ya no hay casos nulos.')
    else:
        print(f'En la columna {col} hay {df.select_dtypes(include=np.number)[col].isnull().sum()} casos nulos.')

En la columna year ya no hay casos nulos.
En la columna age ya no hay casos nulos.


- Reemplazad los valores nulos de la columna sex por la moda, usando el método SimpleImputer.
  
   💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.


In [181]:
#vamos a comprobar cuántos nulos tiene la columna "sex"
print(f'La columna Sexo tiene {df["sex"].isnull().sum()} casos nulos.')
print('-----------------------------------')
#la distribución de hombres y mujeres:
print(f'En la columna Sexo del nuevo DataFrame, tenemos {df["sex"].value_counts()[0]} hombres y {df["sex"].value_counts()[1]} mujeres.')
print('-----------------------------------')
#y calcularemos la moda:
print(f'La moda de la columna Sexo es {df["sex"].mode()[0]} (hombres).')

La columna Sexo tiene 14 casos nulos.
-----------------------------------
En la columna Sexo del nuevo DataFrame, tenemos 1435 hombres y 223 mujeres.
-----------------------------------
La moda de la columna Sexo es M (hombres).


In [182]:
#aplicamos la función
df = simple_imputer(df, "sex", "most_frequent")

In [183]:
#comprobamos los casos perdidos:
df.stb.missing()

,missing,total,percent
month,181,1672,10.825359
species,126,1672,7.535885
fatal,99,1672,5.921053
activity,31,1672,1.854067
country,10,1672,0.598086
year,0,1672,0.000000
type,0,1672,0.000000
age,0,1672,0.000000
sex,0,1672,0.000000


In [184]:
print(f'Después de aplicar el reemplazo de nulos, en la columna Sexo del nuevo DataFrame tenemos {df["sex"].value_counts()[0]} hombres y {df["sex"].value_counts()[1]} mujeres.')

Después de aplicar el reemplazo de nulos, en la columna Sexo del nuevo DataFrame tenemos 1449 hombres y 223 mujeres.


In [185]:
if df["sex"].isnull().sum() == 0:
    print(f'En la columna "Sexo" ya no hay casos nulos.')
else:
    print(f'En la columna "Sexo" hay {df["sex"].isnull().sum()} casos nulos.')

En la columna "Sexo" ya no hay casos nulos.


- Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda) con el método SimpleImputer.

La columna type no tiene valores nulos.

In [186]:
df.stb.freq(['country'], thresh=85, other_label='other countries')

,country,count,percent,cumulative_count,cumulative_percent
0,usa,741,44.584838,741,44.584838
1,australia,343,20.637786,1084,65.222623
2,south africa,234,14.079422,1318,79.302046
3,bahamas,29,1.744886,1347,81.046931
4,new zealand,21,1.263538,1368,82.310469
5,reunion,17,1.022864,1385,83.333333
6,papua new guinea,16,0.962696,1401,84.296029
7,other countries,261,15.703971,1662,100.000000


In [187]:
print(f'La columna Country tiene {df["country"].isnull().sum()} casos nulos.')


La columna Country tiene 10 casos nulos.


In [188]:
#asignamos el valor constante por el que queremos reemplazar los nulos. En este caso, la moda.
imputer_mode_c =  SimpleImputer(strategy='most_frequent', missing_values=np.nan)

In [189]:
#Calculamos cuál es la moda de nuestra columna para comprobar que el reemplazo ha funcionado cuando lo apliquemos:
print(f'La moda de la columna País es {(df["country"].mode()[0]).upper()}.')

La moda de la columna País es USA.


In [190]:
#generamos el array y lo volcamos directamente en el DF:
df_country = pd.DataFrame(imputer_mode_c.fit_transform(df[['country']]), columns=['Pais'])
df_country.head()

,Pais
0,usa
1,brazil
2,usa
3,australia
4,england


In [191]:
#comprobamos que los valores nulos no están presentes en el nuevo DF:
print(f'Hay {df_country["Pais"].isnull().sum()} casos nulos en el nuevo DF.')

Hay 0 casos nulos en el nuevo DF.


In [192]:
#reemplazamos los valores:
df['country'].replace(df['country'].values, df_country['Pais'].values, inplace=True)

#comprobamos que se han aplicado los cambios en los valores de la moda (USA)
df.stb.freq(['country'], thresh=85)

,country,count,percent,cumulative_count,cumulative_percent
0,usa,751,44.916268,751,44.916268
1,australia,343,20.514354,1094,65.430622
2,south africa,234,13.995215,1328,79.425837
3,bahamas,29,1.734450,1357,81.160287
4,new zealand,21,1.255981,1378,82.416268
5,reunion,17,1.016746,1395,83.433014
6,papua new guinea,16,0.956938,1411,84.389952
7,others,261,15.610048,1672,100.000000


In [193]:
if df["country"].isnull().sum() == 0:
    print(f'En la columna "country" ya no hay casos nulos.')
else:
    print(f'En la columna "country" hay {df["country"].isnull().sum()} casos nulos.')

En la columna "country" ya no hay casos nulos.


- Utilizad el método KNN Imputer para reemplazar todos los valores nulos de las columnas numéricas.

Las dos columnas numéricas que tenemos, ya no tienen nulos, por lo que llamamos de vuelta al dataframe original.

In [194]:
#invocamos la copia del df original, y comprobamos que al menos 1 de las cols contiene nulos.
numericas = df2.select_dtypes(include = np.number)
numericas.isnull().sum()

year      0
age     158
dtype: int64

In [195]:
#generamos la función, recogiendo esta vez el parámetro 'vecino':
def knn_imputer(df, vecino):

    imputer = KNNImputer(n_neighbors = vecino)

    imputer.fit(numericas)
  
    df_numericas_trans = pd.DataFrame(imputer.transform(numericas), columns = numericas.columns)

    columnas = df_numericas_trans.columns

    df.drop(columnas, axis = 1, inplace = True)
    
    df[columnas] = df_numericas_trans[columnas]

    return df

In [203]:
dfb = knn_imputer(df2, 5)

In [204]:
dfb.stb.missing()

,missing,total,percent
month,181,1672,10.825359
species,126,1672,7.535885
fatal,99,1672,5.921053
activity,31,1672,1.854067
type,0,1672,0.000000
country,0,1672,0.000000
sex,0,1672,0.000000
year,0,1672,0.000000
age,0,1672,0.000000


In [205]:
#comprobamos que ya no existen casos nulos:
dfb_numericas = dfb.select_dtypes(include=np.number)

for col in dfb_numericas:
    if dfb_numericas[col].isnull().sum() != 0:
        print(f'En la columna {col} hay {dfb_numericas[col].isnull().sum()} casos nulos.')
    else:
        print(f'En la columna {col} no hay casos nulos.')

En la columna year no hay casos nulos.
En la columna age no hay casos nulos.


- Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [199]:
df3 = pd.read_csv("datos/Limpieza-3.csv", index_col = 0)

In [209]:
def iterative_imputer(df3, estadis):

    imputer = IterativeImputer(n_nearest_features = None, initial_strategy = estadis, imputation_order = "ascending")

    imputer.fit(numericas)
 
    df_numericas_trans = pd.DataFrame(imputer.transform(numericas), columns = numericas.columns)

    columnas = df_numericas_trans.columns

    df2.drop(columnas, axis = 1, inplace = True)

    df2[columnas] = df_numericas_trans[columnas]

    return df2

In [213]:
df_itim = iterative_imputer(df3, "mean")

In [214]:
df_itim.stb.missing()

,missing,total,percent
month,181,1672,10.825359
species,126,1672,7.535885
fatal,99,1672,5.921053
activity,31,1672,1.854067
type,0,1672,0.000000
country,0,1672,0.000000
sex,0,1672,0.000000
year,0,1672,0.000000
age,0,1672,0.000000


In [215]:
df_itim_num = df_itim.select_dtypes(include=np.number)

for col in df_itim_num:
    if df_itim_num[col].isnull().sum() != 0:
        print(f'En la columna {col} hay {df_itim_num[col].isnull().sum()} casos nulos.')
    else:
        print(f'En la columna {col} no hay casos nulos.')

En la columna year no hay casos nulos.
En la columna age no hay casos nulos.


- ¿Podríais explicar qué diferencia hay entre estos dos últimos métodos?

Tanto el SimpleImputer como el KNNImputer son modelos pasados en métodos de imputación, es decir, métodos que reemplazan los datos nulos al crear una matriz de datos completa a partir de correlaciones entre valores de la base original (conocidos como "vecinos"). 

La **diferencia entre SimpleImputer y KNNImputer** es que ambos usan diferentes estrategias para calcular estos valores por los que reemplazar a los NA:

SimpleImputer reemplaza los NA por valor constante asignado o a través de estadísticos como media, moda (conocido como "más frecuente") o la mediana de cada columna en la que se encuentran los valores perdidos. Sin embargo, KNNImputer realiza un cálculo de manera iterativa a partir de los datos disponibles en otras columnas.

Por ello, KNNImputer es un modelo más complejo, elaborado y matemáticamente más certero que el SimpleImputer.

### 2. Guardad el csv para seguir trabajando con el en los siguientes ejercicios de pair.

In [ ]:
df.to_csv('datos/Limpieza-5.csv')